Created by: [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [Github](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [FBI NICS](https://www.fbi.gov/file-repository/nics_firearm_checks_-_month_year_by_state_type.pdf/view) | [BuzzfeedNews](https://github.com/BuzzFeedNews/nics-firearm-background-checks/blob/master/data/nics-firearm-background-checks.csv).
<hr>

# Rhode Island NICS Background Checks

This Notebook contains code releated to the data cleaning for my [Tableau dashboard](https://public.tableau.com/profile/smirkygraphs#!/vizhome/RhodeIslandNICSBackgroundChecks/main). The code automatically downloads the most recent data from BuzzfeedNew's Github which parses the monthly PDF's from the FBI NICS. The data is shaped for use in Tableau (tall instead of wide) and cleaned up a bit. Additionally at the bottom their is code that will rank all states by their YoY increase and YTD increase. An issue with this is that every state has different laws so comparing is difficult, for example Kentucky does a recheck on every permit every month.

<hr>

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# make file paths
Path("./data/raw").mkdir(parents=True, exist_ok=True)
Path("./data/clean").mkdir(parents=True, exist_ok=True)

# download data from github
file = 'nics-firearm-background-checks/master/data/nics-firearm-background-checks.csv'
df = pd.read_csv(f'https://raw.githubusercontent.com/BuzzFeedNews/{file}')
df.to_csv('./data/raw/nics-firearm-background-checks.csv', index=False)

In [3]:
# load data and filter for rhode island
df = pd.read_csv('./data/raw/nics-firearm-background-checks.csv', parse_dates=['month'])

# melt (pivot) data turning it from wide to long for tableau & remove total
df = df.drop(columns=['totals']).rename(columns={'month':'date'})
df = df.melt(id_vars=['state', 'date'])

# add sales type
types = ['prepawn', 'redemption', 'returned', 'rentals', 'private_sale', 'return_to_seller']
for sale_type in types:
    df.loc[df['variable'].str.startswith(sale_type), 'type'] = sale_type
df['type'] = df['type'].fillna('FFL/LEA')

# remove 1998 as it only has nov/dec
df = df[df['date'].dt.year > 1998]

# add weapon style
df.loc[df['variable'].str.startswith('permit'), 'style'] = 'permit'
df.loc[df['variable'].str.endswith('handgun'), 'style'] = 'handgun'
df.loc[df['variable'].str.endswith('long_gun'), 'style'] = 'long_gun'
df.loc[df['variable'].str.endswith('other'), 'style'] = 'other'
df.loc[df['variable'] == 'multiple', 'style'] = 'multiple'
df.loc[df['variable'] == 'admin', 'style'] = 'admin'

# save rhode island extract
ri = df[df['state']=='Rhode Island']
ri.to_csv('./data/clean/ri-nics-firearm-clean.csv', index=False)

In [4]:
# drop unwanted states & keep states + dc
drop = ['Guam', 'Mariana Islands', 'Puerto Rico', 'Virgin Islands']
states = [x for x in df['state'].unique().tolist() if x not in drop]
df = df[df['state'].isin(states)]

# save all states data
df.to_csv('./data/clean/nics-firearm-clean.csv', index=False)

# get percent change from prior year (full and ytd)
def full_ranks(df):
    df = df.groupby([df['state'], df['date'].dt.year]).sum()
    df['pct'] = df.pct_change()
    
    df = df.rename(columns={'value': 'fy_value', 'pct': 'fy_pct'})
    
    return df

def ytd_ranks(df):
    # filter all years to current month
    df = df[df['date'].dt.month <= df['date'].max().month]
    df = df.groupby([df['state'], df['date'].dt.year]).sum()
    df['pct'] = df.pct_change()
    
    df = df.rename(columns={'value': 'ytd_value', 'pct': 'ytd_pct'})
    
    return df

# combine and save
fy_df = full_ranks(df)
ytd_df = ytd_ranks(df)
df = pd.concat([fy_df, ytd_df], axis=1).reset_index()
df.to_csv('./data/clean/state-ranks.csv', index=False)